In [6]:
#Document Structure
from langchain_core.documents import Document

In [7]:
doc=Document(
    page_content="this is the main text content I am using to create RAG",
    metadata={
        "source":"examples.txt",
        "pages":1,
        "author":"Krish Naik",
        "date_created":"2025-1-1"
    }
)
doc

Document(metadata={'source': 'examples.txt', 'pages': 1, 'author': 'Krish Naik', 'date_created': '2025-1-1'}, page_content='this is the main text content I am using to create RAG')

In [8]:
import os
os.makedirs("../data/text_files",exist_ok=True)

In [9]:
sample_texts={
    "../data/text_files/intro.txt":"""
        Python is a high-level, interpreted programming language known for its simplicity and readability.
Created by Guido van Rossum and first released in 1991, Python has become one of the most popular
programming languages in the world.

Key Features:
- Easy to learn and use
- Extensive standard library
- Cross-platform compatibility
- Strong community support

Python is widely used in web development, data science, artificial intelligence, and automation.""",
    
    "../data/text_files/machine_learning.txt": """Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables systems to learn and improve
from experience without being explicitly programmed. It focuses on developing computer programs
that can access data and use it to learn for themselves.

Types of Machine Learning:
1. Supervised Learning: Learning with labeled data
2. Unsupervised Learning: Finding patterns in unlabeled data
3. Reinforcement Learning: Learning through rewards and penalties

Applications include image recognition, speech processing, and recommendation systems
    
"""
}
for filepath,content in sample_texts.items():
    with open(filepath,"w",encoding="utf-8") as f:
        f.write(content)

print("sample files is created")

sample files is created


In [10]:
from langchain_community.document_loaders import TextLoader

In [11]:
loader=TextLoader("../data/text_files/intro.txt",encoding="utf-8")
document=loader.load()
print(document)

[Document(metadata={'source': '../data/text_files/intro.txt'}, page_content='\n        Python is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popular\nprogramming languages in the world.\n\nKey Features:\n- Easy to learn and use\n- Extensive standard library\n- Cross-platform compatibility\n- Strong community support\n\nPython is widely used in web development, data science, artificial intelligence, and automation.')]


In [12]:
from langchain_community.document_loaders import DirectoryLoader

dir_loader=DirectoryLoader(
    "../data/text_files",
    glob="**/*.txt",
    loader_cls=TextLoader,
    loader_kwargs={'encoding':'utf-8'},
    show_progress=False 
)
documents=dir_loader.load()
print(documents)

[Document(metadata={'source': '../data/text_files/intro.txt'}, page_content='\n        Python is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popular\nprogramming languages in the world.\n\nKey Features:\n- Easy to learn and use\n- Extensive standard library\n- Cross-platform compatibility\n- Strong community support\n\nPython is widely used in web development, data science, artificial intelligence, and automation.'), Document(metadata={'source': '../data/text_files/machine_learning.txt'}, page_content='Machine Learning Basics\n\nMachine learning is a subset of artificial intelligence that enables systems to learn and improve\nfrom experience without being explicitly programmed. It focuses on developing computer programs\nthat can access data and use it to learn for themselves.\n\nTypes of Machine Learning:\n1. Supervised Learning: Learning with labeled

In [13]:
from langchain_community.document_loaders import PyPDFLoader,PyMuPDFLoader
dir_loader=DirectoryLoader(
    "../data/text_files",
    glob="**/*.pdf",
    loader_cls=PyPDFLoader,
    show_progress=False 
)
pdf_documents=dir_loader.load()
print(pdf_documents)

[Document(metadata={'producer': 'PyPDF2', 'creator': 'PyPDF', 'creationdate': '', 'subject': 'Neural Information Processing Systems http://nips.cc/', 'publisher': 'Curran Associates, Inc.', 'language': 'en-US', 'created': '2017', 'eventtype': 'Poster', 'description-abstract': 'The dominant sequence transduction models are based on complex recurrent orconvolutional neural networks in an encoder and decoder configuration. The best performing such models also connect the encoder and decoder through an attentionm echanisms.  We propose a novel, simple network architecture based solely onan attention mechanism, dispensing with recurrence and convolutions entirely.Experiments on two machine translation tasks show these models to be superiorin quality while being more parallelizable and requiring significantly less timeto train. Our single model with 165 million parameters, achieves 27.5 BLEU onEnglish-to-German translation, improving over the existing best ensemble result by over 1 BLEU. On 

In [14]:
type(pdf_documents[0])

langchain_core.documents.base.Document

In [15]:
len(pdf_documents)

11

In [16]:
#vectorstore and embeddings
import numpy as np
from sentence_transformers import SentenceTransformer
# import chromadb
# from chromadb.config import Settings
import uuid
from typing import List,Dict,Any,Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [77]:
%pip install chromadb


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [56]:
%pip install sentence-transformers


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [17]:
#vectorstore and embeddings
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
# from chromadb.config import Settings
import uuid
from typing import List,Dict,Any,Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
class EmbeddingManager:
    def __init__(self,model_name:str="all-MiniLM-L6-v2"):
        self.model_name=model_name
        self.model=None
        self._load_model()

    def _load_model(self):
        try:
            print("loading embedding model...",self.model_name)
            self.model=SentenceTransformer(self.model_name)
            print("model loaded successfully and embedding dimension is ",self.model.get_sentence_embedding_dimension())
        except Exception as e:
            print("error loading embedding model",e)
            raise 
    
    def generate_embeddings(self,texts:List[str])->np.ndarray:
        if not self.model:
            raise ValueError("embedding model no loaded")
        print(f"generating embeddings for {len(texts)} texts... ")
        embeddings=self.model.encode(texts,show_progress_bar=True,normalize_embeddings=True)
        print(f"generated embeddings with shape {embeddings.shape}")
        return embeddings
    
## initialize the embeddings manager
embedding_manager=EmbeddingManager()
embedding_manager

loading embedding model... all-MiniLM-L6-v2


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 2340.48it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


model loaded successfully and embedding dimension is  384


In [8]:
%pip upgrade chromadb

ERROR: unknown command "upgrade"
Note: you may need to restart the kernel to use updated packages.


In [19]:
#vectorstore
from typing import List,Dict,Any,Tuple
import chromadb
import numpy as np
import os
from chromadb.config import Settings

class VectorStore:
    def __init__(self,collection_name:str="pdf_documents",persist_directory:str="../data/vector_store"):
        self.collection_name=collection_name
        self.persist_directory=persist_directory
        self.client=None
        self.collection=None
        self._initialize_vector_store()
    def _initialize_vector_store(self):
        try:
            os.makedirs(self.persist_directory,exist_ok=True)
            self.client=chromadb.PersistentClient(path=self.persist_directory)
            self.collection=self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"hnsw:space":"cosine"}
            )
            print(f"vector store initialized collection :{self.collection_name}")
            print(f"existing documents in collection :{self.collection.count()}")
        except Exception as e:
            print("error initializing vector store",e)
            raise
    
    def add_documents(self,documents:List[Any],embeddings:np.ndarray):
        if len(documents)!=len(embeddings):
            raise ValueError("number of documents must match number of embeddings")
        
        print(f"adding {len(documents)} docuements to vector store...")

        ids=[]
        metadata_list=[]
        document_text=[]
        embedding_list=[]
        for i,(doc,embedding) in enumerate(zip(documents,embeddings)):
            doc_id=f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)

            metadata=dict(doc.metadata)
            metadata['doc_index']=i
            metadata['content_length']=len(doc.page_content)
            metadata_list.append(metadata)

            document_text.append(doc.page_content)

            embedding_list.append(embedding.tolist())
        try:
            self.collection.add(
                ids=ids,
                embeddings=embedding_list,
                metadatas=metadata_list,
                documents=document_text
            )
            print(f"successfully added {len(documents )} documents to vector store")
            print(f"total documents in collection :{self.collection.count()}")

        except Exception as e:
            print(f"error adding documents to vector store",{e})
            raise
                                           
## initialize vector store
vector_store=VectorStore()
vector_store

vector store initialized collection :pdf_documents
existing documents in collection :0


In [20]:
texts=[doc.page_content for doc in pdf_documents]
texts

['Attention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗†\nUniversity of Toronto\naidan@cs.toronto.edu\nŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation tasks show these models to\nbe superior in quality while being more parallelizable and requiring signiﬁcantl

In [21]:
##generate the embeddings 
embeddings=embedding_manager.generate_embeddings(texts)

#store in vector database 
vector_store.add_documents(pdf_documents,embeddings)



generating embeddings for 11 texts... 


Batches: 100%|██████████| 1/1 [00:00<00:00,  4.20it/s]

generated embeddings with shape (11, 384)
adding 11 docuements to vector store...
successfully added 11 documents to vector store
total documents in collection :11


In [29]:
#retriver for rag
class RagRetriever:
    def __init__(self,vector_store:VectorStore,embedding_manager:EmbeddingManager):
        self.vector_store=vector_store
        self.embedding_manager=embedding_manager

    def retrieve(self,query:str,top_k:int=5,score_threshold:float=0.2,max_distance:float=0.8)->List[Dict[str,Any]]:
        print(f"retrieving documents for query: {query}")
        print(f"top k {top_k} , score threshold {score_threshold}")
         
        query_embedding=self.embedding_manager.generate_embeddings([query])[0]

        try:
            results=self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            retrieved_docs=[]
            if results['documents'] and results['documents'][0]:
                documents=results['documents'][0]
                metadatas=results['metadatas'][0]
                distances=results['distances'][0]
                ids=results['ids'][0]

                for i,(doc,document,metadata,distance) in  enumerate(zip(ids,documents,metadatas,distances)):
                    similarity_score=1-distance
                    if distance<=max_distance:
                        retrieved_docs.append({
                            'id':doc,
                            'content':document,
                            'metadata':metadata,
                            'similarity_score':similarity_score,
                            'distance':distance,
                            'rank':i+1
                        })

                    print(f"retrieved {len(retrieved_docs)} documents after filtering")
            else:
                print("no docs found")
            
            return retrieved_docs 
        except Exception as e:
            print("error retrieving documents from vector store",e)
            return []
        
rag_retriever=RagRetriever(vector_store,embedding_manager)


In [30]:
rag_retriever

In [31]:
rag_retriever.retrieve("What is attention is all you need")

retrieving documents for query: What is attention is all you need
top k 5 , score threshold 0.2
generating embeddings for 1 texts... 


Batches: 100%|██████████| 1/1 [00:00<00:00,  3.80it/s]

generated embeddings with shape (1, 384)
retrieved 1 documents after filtering
retrieved 2 documents after filtering
retrieved 3 documents after filtering
retrieved 4 documents after filtering
retrieved 5 documents after filtering


[{'id': 'doc_31c5d074_3',
  'content': 'Scaled Dot-Product Attention\n Multi-Head Attention\nFigure 2: (left) Scaled Dot-Product Attention. (right) Multi-Head Attention consists of several\nattention layers running in parallel.\nquery with all keys, divide each by√dk, and apply a softmax function to obtain the weights on the\nvalues.\nIn practice, we compute the attention function on a set of queries simultaneously, packed together\ninto a matrixQ. The keys and values are also packed together into matricesK andV . We compute\nthe matrix of outputs as:\nAttention(Q,K,V ) = softmax(QK T\n√dk\n)V (1)\nThe two most commonly used attention functions are additive attention [2], and dot-product (multi-\nplicative) attention. Dot-product attention is identical to our algorithm, except for the scaling factor\nof 1√dk\n. Additive attention computes the compatibility function using a feed-forward network with\na single hidden layer. While the two are similar in theoretical complexity, dot-product

In [25]:
import os
import chromadb

def _initialize_vector_store(self):
    try:
        abs_path = os.path.abspath(self.persist_directory)
        print("VECTOR STORE PATH:", abs_path)

        os.makedirs(abs_path, exist_ok=True)

        self.client = chromadb.PersistentClient(path=abs_path)

    except Exception as e:
        print("error initializing vector store", e)
        raise


In [33]:
vector_store=VectorStore()

vector store initialized collection :pdf_documents
existing documents in collection :11


In [39]:
#simple rag pipeline with groq llm
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()

#initialize the groq llm
groq_api_key=os.getenv("GROQ_API_KEY")
llm=ChatGroq(api_key=groq_api_key,model="llama-3.1-8b-instant",temperature=0.1,max_tokens=1024)

#simple rag function :retrieve context and generate response
def rag_chain(query,retriever,llm,top_k=3):
    ##retrieve the context
    results=retriever.retrieve(query,top_k=top_k)
    context="\n\n".join([doc['content']for doc in results]) if results else ""
    if not context:
        return "No relevant context found to answer the question"
    
    #generate the anwer using groq llm
    prompt=f"""Use the following context to answer the question concisely.
    Instructions:
    - Answer based on the retrieved documents only
    - If the answer is not found in the context, say "I don't know"
    - Answer Professionally and precisely
    Context":{context}
    Question: {query}
    Answer:"""

    response=llm.invoke([prompt.format(context=context,query=query)])
    return response.content


python-dotenv could not parse statement starting at line 1


In [40]:
answer=rag_chain("what is attention mechanism?",rag_retriever,llm)
print(answer)

retrieving documents for query: what is attention mechanism?
top k 3 , score threshold 0.2
generating embeddings for 1 texts... 


Batches: 100%|██████████| 1/1 [00:00<00:00,  5.50it/s]

generated embeddings with shape (1, 384)
retrieved 1 documents after filtering
retrieved 2 documents after filtering
retrieved 3 documents after filtering


An attention mechanism is a method used in deep learning models to allow the model to focus on specific parts of the input data when generating the output. It allows the model to weigh the importance of different input elements and select the most relevant ones to use when making predictions.


In [49]:
def enhanced_rag(query,retriever,llm,top_k=5,min_score=0.2,return_context=False):
    results=retriever.retrieve(query,top_k=top_k,score_threshold=min_score)
    if not results:
        return {
            "answer":"no relevant context found",
            'sources':[],
            "confidence":0.0,
            "context":"" if return_context else None
        }
    context="\n\n".join([doc['content'] for doc in results])
    sources=[{
        'source':doc['metadata'].get('source_file',doc['metadata'].get('source','unknown')),
        'page':doc['metadata'].get('page','unknown'),
        'score':doc['similarity_score'],
        'preview':doc['content'][:300]+"..."
    }for doc in results]
    confidence=max([doc['similarity_score'] for doc in results])
    prompt=f"""use the following context to answer the question concisely.\n Context:\n{ context}\n\nQuestion:{query}\n\nAnswer:"""
    response=llm.invoke([prompt.format(context=context,query=query)])
    output={
        "answer":response.content,
        "sources":sources,
        'confidence':confidence
    }
    if return_context:
        output['context']=context

    return output

In [50]:
print(type(result))

<class 'str'>


In [54]:
result=enhanced_rag("What can we conclude from this paper?",rag_retriever,llm,top_k=3,min_score=0.1,return_context=True)
print("Answer ",result['answer'])
print("Sources ",result['sources'])
print("Confidence Score ",result['confidence'])
print('Context Preview ',result['context'][:300])

retrieving documents for query: What can we conclude from this paper?
top k 3 , score threshold 0.1
generating embeddings for 1 texts... 


Batches: 100%|██████████| 1/1 [00:00<00:00, 13.00it/s]

generated embeddings with shape (1, 384)
retrieved 1 documents after filtering
retrieved 1 documents after filtering
retrieved 1 documents after filtering


Answer  The paper concludes that the Transformer, a sequence transduction model based entirely on attention, achieves a new state of the art in translation tasks, particularly in the English-to-German and English-to-French translation tasks. It also highlights the potential of attention-based models for future research, including applying them to tasks involving input and output modalities other than text, and investigating local attention mechanisms for handling large inputs and outputs.
Sources  [{'source': '../data/text_files/attention is all you need-1.pdf', 'page': 8, 'score': 0.20160531997680664, 'preview': 'Table 3: Variations on the Transformer architecture. Unlisted values are identical to those of the base\nmodel. All metrics are on the English-to-German translation development set, newstest2013. Listed\nperplexities are per-wordpiece, according to our byte-pair encoding, and should not be compared to...'}]
Confidence Score  0.20160531997680664
Context Preview  Table 3: Varia

In [58]:
from typing import List, Dict, Any
import time

class AdvancedRAGPipeline:
    def __init__(self, retriever, llm):
        self.retriever = retriever
        self.llm = llm
        self.history = []  # Store query history

    def query(self, question: str, top_k: int = 5, min_score: float = 0.2, stream: bool = False, summarize: bool = False) -> Dict[str, Any]:
        # Retrieve relevant documents
        results = self.retriever.retrieve(question, top_k=top_k, score_threshold=min_score)
        if not results:
            answer = "No relevant context found."
            sources = []
            context = ""
        else:
            context = "\n\n".join([doc['content'] for doc in results])
            sources = [{
                'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
                'page': doc['metadata'].get('page', 'unknown'),
                'score': doc['similarity_score'],
                'preview': doc['content'][:120] + '...'
            } for doc in results]
            # Streaming answer simulation
            prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {question}\n\nAnswer:"""
            if stream:
                print("Streaming answer:")
                for i in range(0, len(prompt), 80):
                    print(prompt[i:i+80], end='', flush=True)
                    time.sleep(0.05)
                print()
            response = self.llm.invoke([prompt.format(context=context, question=question)])
            answer = response.content

        # Add citations to answer
        citations = [f"[{i+1}] {src['source']} (page {src['page']})" for i, src in enumerate(sources)]
        answer_with_citations = answer + "\n\nCitations:\n" + "\n".join(citations) if citations else answer

        # Optionally summarize answer
        summary = None
        if summarize and answer:
            summary_prompt = f"Summarize the following answer in 2 sentences:\n{answer}"
            summary_resp = self.llm.invoke([summary_prompt])
            summary = summary_resp.content

        # Store query history
        self.history.append({
            'question': question,
            'answer': answer,
            'sources': sources,
            'summary': summary
        })

        return {
            'question': question,
            'answer': answer_with_citations,
            'sources': sources,
            'summary': summary,
            'history': self.history
        }

# Example usage:
adv_rag = AdvancedRAGPipeline(rag_retriever, llm)
result = adv_rag.query("what is a decoder in model Architecture?", top_k=3, min_score=0.1, stream=True, summarize=True)
print("\nFinal Answer:", result['answer'])
print("Summary:", result['summary'])
print("History:", result['history'][-1])

retrieving documents for query: what is a decoder in model Architecture?
top k 3 , score threshold 0.1
generating embeddings for 1 texts... 


Batches: 100%|██████████| 1/1 [00:00<00:00, 12.33it/s]

generated embeddings with shape (1, 384)
retrieved 1 documents after filtering
retrieved 2 documents after filtering
retrieved 3 documents after filtering
Streaming answer:
Use the following context to answer the question concisely.
Context:
Figure 1: The Transformer - model architecture.
wise fully connected feed-forward network. We employ a residual connection [10] around each of
the two sub-layers, followed

 by layer normalization [ 1]. That is, the output of each sub-layer is
LayerNorm(x + Sublayer(x)), where Sublayer(x) is the function implemented by the sub-layer
itself. To facilitate these residual connections, all sub-layers in the model, as well as the embedding
layers, produce outputs of dimensiondmodel = 512.
Decoder: The decoder is also composed of a stack ofN = 6 identical layers. In addition to the two
sub-layers in each encoder layer, the decoder inserts a third sub-layer, which performs multi-head
attention over the output of the encoder stack. Similar to the encoder, we employ residual connections
around each of the sub-layers, followed by layer normalization. We also modify the self-attention
sub-layer in the decoder stack to prevent positions from attending to subsequent positions. This
masking, combined with fact that the output embeddings are offset by one position, ensures that the
predictions for positioni can depend only on the known outputs at positions less thani.
3